# Swarms

Code examples from [Think Complexity, 2nd edition](https://thinkcomplex.com).

Copyright 2016 Allen Downey, [MIT License](http://opensource.org/licenses/MIT)

This is an adaptation of Allen Downey's `Boids7.py` to the Jupyter Notebook.

Note that `vpython` will not run on Python 3.13.

In [5]:
""" Code example from Think Complexity, by Allen Downey.

Original code by Matt Aasted, modified by Allen Downey.

Based on Reynolds, "Flocks, Herds and Schools" and
Flake, "The Computational Beauty of Nature."

Copyright 2011 Allen B. Downey.
Distributed under the MIT License.
"""
try:
    from vpython import *
except:
    print("This program requires VPython 7, which you can read about")
    print("at http://vpython.org/.  If you are using Anaconda, you can")
    print("install VPython by running the following on the command line:")
    print("conda install -c vpython vpython")
    raise ImportError

scene = canvas() 
import numpy as np

<IPython.core.display.Javascript object>

Note that the canvas will be placed immediately above this cell (i.e. you may need to scroll up after executing the cells below).

In [6]:
null_vector = vector(0,0,0)


def random_vector(a, b):
    """Create a vector with each element uniformly distributed in [a, b)."""
    coords = np.random.uniform(a, b, size=3)
    return vector(*coords)


def limit_vector(vect):
    """If the magnitude is greater than 1, set it to 1"""
    if vect.mag > 1:
        vect.mag = 1
    return vect


class Boid(cone):
    """A Boid is a VPython cone with a velocity and an axis, belonging to a swarm."""

    def __init__(self, radius=0.03, length=0.1, swarm_id=0):
        self.swarm_id = swarm_id  # Assign a swarm ID to the boid
        pos = random_vector(0, 1)
        self.vel = random_vector(0, 1).norm()

        # Assign a unique color based on swarm_id
        colors = [vector(1, 0, 0), vector(0, 1, 0), vector(0, 0, 1), vector(1, 1, 0)]
        color_choice = colors[swarm_id % len(colors)]  

        cone.__init__(self, pos=pos, radius=radius, length=length, color=color_choice)
        self.axis = length * self.vel

    def get_neighbors(self, boids, radius, angle):
        """Return a list of neighbors within a field of view, limited to same swarm."""
        neighbors = []
        for boid in boids:
            if boid is self:
                continue

            # Ensure only Boids are checked for swarm_id
            if isinstance(boid, Boid) and boid.swarm_id != self.swarm_id:
                continue

            offset = boid.pos - self.pos
            if offset.mag > radius:
                continue

            diff = self.vel.diff_angle(offset)
            if abs(diff) > angle:
                continue

            neighbors.append(boid)

        return neighbors


    def center(self, boids, radius=1, angle=1):
        """Find the center of mass of other boids in range and
        return a vector pointing toward it."""
        neighbors = self.get_neighbors(boids, radius, angle)
        vecs = [boid.pos for boid in neighbors]
        return self.vector_toward_center(vecs)

    def vector_toward_center(self, vecs):
        """Vector from self to the mean of vecs.

        vecs: sequence of vector

        returns: Vector
        """
        if vecs:
            center = np.mean(vecs)
            toward = vector(center - self.pos)
            return limit_vector(toward)
        else:
            return null_vector

    def avoid(self, boids, carrot, radius=0.8, angle=np.pi):
        """Avoids nearby boids, with stronger repulsion from different swarms."""
        objects = [boid for boid in boids if hasattr(boid, 'swarm_id')]  # Exclude non-boids
        neighbors = self.get_neighbors(objects, radius, angle)

        # Separate swarm and non-swarm neighbors
        same_swarm = [boid for boid in neighbors if boid.swarm_id == self.swarm_id]
        other_swarms = [boid for boid in neighbors if boid.swarm_id != self.swarm_id]

        # Stronger attraction within the same swarm
        same_swarm_avoidance = -0.5 * self.vector_toward_center([boid.pos for boid in same_swarm])

        # STRONGER avoidance from other swarms
        other_swarm_avoidance = -5 * self.vector_toward_center([boid.pos for boid in other_swarms])

        return limit_vector(same_swarm_avoidance + other_swarm_avoidance)


    def align(self, boids, radius=0.5, angle=1):
        """Return the average heading of other boids in range.

        boids: list of Boids
        """
        neighbors = self.get_neighbors(boids, radius, angle)
        vecs = [boid.vel for boid in neighbors]
        return self.vector_toward_center(vecs)

    def love(self, carrot):
        """Returns a vector pointing toward the carrot."""
        toward = carrot.pos - self.pos
        return limit_vector(toward)

    def set_goal(self, boids, carrot):
        """Sets movement goals based on swarm dynamics."""

        # General weights (default)
        w_avoid = 5
        w_center = 3
        w_align = 2
        w_love = 5

        # Adjust behavior per swarm
        if self.swarm_id == 0:  
            w_avoid, w_center, w_align, w_love = 6, 10, 6, 3
        elif self.swarm_id == 1: 
            w_avoid, w_center, w_align, w_love = 6, 10, 6, 3
        elif self.swarm_id == 2:  
            w_avoid, w_center, w_align, w_love = 6, 10, 6, 3

        self.goal = (w_center * self.center(boids) +
                     w_avoid * self.avoid(boids, carrot) +
                     w_align * self.align(boids) +
                     w_love * self.love(carrot))

        self.goal.mag = 1


    def move(self, mu=0.1, dt=0.1):
        """Update the velocity, position and axis vectors.

        mu: how fast the boids can turn (maneuverability).
        dt: time step
        """

        self.vel = (1-mu) * self.vel + mu * self.goal
        self.vel.mag = 1
        self.pos += dt * self.vel
        self.axis = self.length * self.vel


class World:
    def __init__(self, n=30, num_swarms=3):
        """Create multiple swarms, spaced apart to start."""

        self.boids = []
        for i in range(n):
            swarm_id = i % num_swarms  # Assign to swarms

            # Separate swarm start positions
            swarm_offset = vector(swarm_id * 2, swarm_id * 2, 0)

            boid = Boid(swarm_id=swarm_id)
            boid.pos += swarm_offset  # Move each swarm to different areas
            self.boids.append(boid)

        self.carrot = sphere(pos=vector(1, 0, 0), radius=0.1, color=vector(1, 0, 0))
        self.tracking = False 


    def step(self):
        """Compute one time step."""
        # move the boids
        for boid in self.boids:
            boid.set_goal(self.boids, self.carrot)
            boid.move()

        # if we're tracking, move the carrot
        if self.tracking:
            self.carrot.pos = scene.mouse.pos

In [7]:
n = 50  # Total boids
num_swarms = 5  # Number of swarms

world = World(n, num_swarms)
scene.center = world.carrot.pos
scene.autoscale = False

while 1:
    rate(30)
    world.step()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 